In [1]:
import numpy as np
from tqdm import tqdm
import cv2
import os
from glob import glob

In [2]:
def dice_coef(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection + 1.) / (np.sum(y_true) + np.sum(y_pred) + 1.)

def iou_coef(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred) - intersection
    return (intersection + 1.) / (union + 1.)

In [8]:
val_gt_mask = glob("/home/mykang/mmsegmentation/data/Satellite/ann_dir/val_slice/*.png")

dice_list = []
iou_list = []

for filename in tqdm(val_gt_mask):
    gt_mask_path = f"/home/mykang/mmsegmentation/data/Satellite/ann_dir/val_slice/{os.path.basename(filename)}"
    pred_mask_path = f"/home/mykang/mmsegmentation/_satellite/deeplabv3plus/format_results/{os.path.basename(filename)}"


    gt_mask = cv2.imread(gt_mask_path, cv2.IMREAD_GRAYSCALE)
    pred_mask = cv2.imread(pred_mask_path, cv2.IMREAD_GRAYSCALE)

    # my inference value is [7,8]
    pred_mask = pred_mask - 7

    # Ignore pairs where there is no building in gt and pred_mask
    gt_sum = np.sum(gt_mask == 1)
    pred_sum = np.sum(pred_mask == 1)
    if gt_sum == 0 and pred_sum == 0:
        continue

    if gt_sum == 0 and pred_sum != 0:
        dice_list.append(0)
        iou_list.append(0)

    iou = iou_coef(gt_mask, pred_mask)
    dice = dice_coef(gt_mask, pred_mask)

    dice_list.append(dice)
    iou_list.append(iou)



print("mIoU", sum(iou_list) / len(iou_list))
print("mDice", sum(dice_list) / len(dice_list))

100%|██████████| 17136/17136 [00:41<00:00, 411.21it/s]

mIoU 0.7350377057451759
mDice 0.8167588074908287
